# Variational Autoencoder

## Prerequirements

In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Keras functions
from keras.models import Sequential, Model
from keras.layers import Input, Lambda, concatenate
from keras.layers import Dense
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2DTranspose, UpSampling2D
from keras.optimizers import SGD, Adam
from keras import metrics

# Keras dataset
from keras.datasets import mnist

# Keras utilis function
from keras.utils import np_utils

from keras import backend as K

Using TensorFlow backend.


## Load MNIST Datasets

In [3]:
(x_train0, y_train0), (x_test0, y_test0) = mnist.load_data()

Reshape and normalize the data.

In [4]:
x_train = x_train0.reshape(60000, -1)
x_test = x_test0.reshape(10000, -1)

x_train -= x_train.min()
x_train = x_train/x_train.max()

## Build VAE

Define the Encoder and Decoder.

In [5]:
enc_1 = Dense(100, activation='sigmoid')
# enc_2 = Dense(2, activation='sigmoid')

enc_mean = Dense(2)
enc_log_var = Dense(2)

dec_2 = Dense(100, activation='sigmoid')
dec_1 = Dense(784, activation='sigmoid')

In [6]:
x = Input(shape=(784,))
enc_x = enc_1(x)

z_mean = enc_mean(enc_x)
z_log_var = enc_log_var(enc_x)

Define the sampling function.

In [7]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(2,), mean=0., stddev=1)
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [8]:
z = Lambda(sampling, output_shape=(2,))([z_mean, z_log_var])

In [9]:
dec_x = dec_2(z)
x_hat = dec_1(dec_x)

In [10]:
VAE = Model(x, x_hat)
VAE.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 100)          78500       input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 2)            202         dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 2)            202         dense_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (

Arbitrary loss function.

In [11]:
def vae_loss(x, x_recon):  
    recovery_loss = 784 * metrics.binary_crossentropy(x, x_recon)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return recovery_loss + kl_loss

Compile VAE.

In [12]:
VAE.compile(loss=vae_loss, optimizer=Adam())

Load the weights of file `VAE_handwriting_model_weights.h5`.

In [13]:
VAE.load_weights('NCTS_model/VAE_handwriting_model_weights.h5')

## VAE Encoder

In [14]:
VAE_Encoder = Model(x, z_mean)
VAE_Encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 78,702
Trainable params: 78,702
Non-trainable params: 0
_________________________________________________________________


## VAE Decoder

In [15]:
h_input = Input(shape=(2,))
VAE_Decoder = Model(h_input, dec_1(dec_2(h_input)))
VAE_Decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               300       
_________________________________________________________________
dense_5 (Dense)              (None, 784)               79184     
Total params: 79,484
Trainable params: 79,484
Non-trainable params: 0
_________________________________________________________________


In [16]:
idx = np.random.randint(x_train.shape[0])
print("第 %d 圖的 latent 表示為 %s" %(idx, VAE_Encoder.predict(x_train[idx: idx+1])))

第 10317 圖的 latent 表示為 [[-0.35468933 -1.3876125 ]]


## Visualize VAE Encoder

In [17]:
def Pick(nums):
    num1 = int(nums_list[0])
    num2 = int(nums_list[1])
    subset1 = np.where(y_train0 == num1)
    ind1 = np.random.choice(subset1[0]) 
    subset2 = np.where(y_train0 == num2)
    ind2 = np.random.choice(subset2[0]) 
    ind = [ind1,ind2]
    return (ind)

In [19]:
def Visualize(w):
    idx = index
    ind1 = idx[0]
    ind2 = idx[1]
    plot_1 = x_train0[ind1]  # x_train0.shape = (28,28)
    plot_2 = x_train0[ind2]
    plot_w = (1-w)*x_train0[ind1] + w*x_train0[ind2]

    mu_1 = VAE_Encoder.predict(x_train[ind1:ind1+1]).squeeze()
    mu_2 = VAE_Encoder.predict(x_train[ind2:ind2+1]).squeeze()
    mu_w = (1-w)*mu_1 + w*mu_2

    plt.figure(figsize=(6, 8))
    ax1 = plt.subplot(2,1,1)
    ax1.set_title('Visualization of VAE Encoder')
    ax1.set_xlim([-4, 4])
    ax1.set_ylim([-4, 4])
     
    # Add a line
    delta_y = mu_2[1] - mu_1[1]
    delta_x = mu_2[0] - mu_1[0]
    slope = delta_y/delta_x
    x = np.linspace(mu_1[0],mu_2[0],50)
    b = mu_1[1] - slope*mu_1[0]
    y = b + slope*x
    ax1.plot(x,y, color='black')
    
    ax1.scatter(mu_1[0], mu_1[1], color='red')
    ax1.scatter(mu_2[0], mu_2[1], color='blue')
    ax1.scatter(mu_w[0], mu_w[1], color='purple')

    ax2 = plt.subplot(2, 3, 4)
    ax2.imshow(plot_2, cmap='Greys')
    ax2.set_title('Blue dot')
    
    ax3 = plt.subplot(2, 3, 5)
    ax3.imshow(plot_w, cmap='Greys')
    ax3.set_title('Purple dot')

    ax4 = plt.subplot(2, 3, 6)
    ax4.imshow(plot_1, cmap='Greys')
    ax4.set_title('Red dot');

In [32]:
x, y = input("Enter two numbers (from 0 to 9): ").split() 
nums_list = [x,y]
index = Pick(nums_list)
print("w: weight")
interact(Visualize, w=FloatSlider(value=0.5, min=0, max=1.0, step=0.05));

Enter two numbers (from 0 to 9): 5 7
w: weight


interactive(children=(FloatSlider(value=0.5, description='w', max=1.0, step=0.05), Output()), _dom_classes=('w…